In [193]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Activation, merge, Input, Lambda, Reshape
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, BatchNormalization
from keras.utils.np_utils import to_categorical
from keras import initializers
from keras import backend as K
from keras.engine.topology import Layer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

import pandas as pd
import numpy as np

In [194]:
import sys
sys.path.insert(0, '/Users/xiaochenzhang/Desktop/PersonalProjects/nlp_NN/')

In [195]:
from tfidf_folder.tfidf import get_instance_tfidf_vector

# Import data
* take first 10000 training data
* take first 4000 testing data

In [ ]:
train_data = pd.read_csv('data/4w_trainset.csv', encoding='GB18030')
test_data = pd.read_csv('data/4k_testset.csv', encoding='GB18030')
train_data = train_data[:10000]
test_data = test_data[:4000]

In [ ]:
test_data.head()

In [79]:
train_data.head()

,ID,工单编号,行业分类1级,行业分类2级,行业分类3级,行业分类4级,诉求内容,原处置单位,处置单位,单位类别
0,6,20180102124706,司法行政,其他,其他,其他,市民来电咨询: 万绿园政府服务中心五楼的消防窗口的电话。,市公安局消防支队,公安局消防支队,3
1,7,20180101115513,其他,其他,其他,其他,市民来电咨询：驾驶证过期换证的问题,市公安局交通警察支队,公安局交通警察支队,3
2,15,20180117311266,公安（治安、交通）,其他,其他,其他,市民微信反映: 琼山区下坎西球场对面有人在聚众赌博（旁边有个停车场），请市公安局核实处理。（...,市公安局,公安局,3
3,17,20180109210991,“12345”热线,传真信息,信息咨询,信息咨询,市民来电咨询:咨询驾管所电话。,市公安局交通警察支队,公安局交通警察支队,3
4,21,20180101118432,城乡建设,道路维护,路面养护,路面污染,市民来电反映:在美兰区灵山镇往文昌方向的琼文大道机场路段每天都有工地车辆洒落泥土在路上，请城...,美兰区城管局,城市管理行政执法局,2


## tfidf convert

In [80]:
train = pd.DataFrame({
    'ID': train_data['ID'],
    'Label': train_data[train_data.columns[-1]],
    'Feature': train_data['ID'].apply(lambda x: get_instance_tfidf_vector(str(x), False))
})

In [82]:
print(train.shape)
train.head(2)

(10000, 3)


,ID,Label,Feature
0,6,3,"[7.345027488669131, 2.1331498953028616, 1.6502..."
1,7,3,"[0.0, 0.0, 0.0, 2.8212059778186624, 3.14354571..."


In [ ]:
test = pd.DataFrame({
    'ID': test_data['ID'],
    'Label': test_data[test_data.columns[-1]],
    'Feature': test_data['ID'].apply(lambda x: get_instance_tfidf_vector(str(x), False))
})

In [ ]:
print(test.shape)
test.head()

## Explore the test dataset
* find a lot of all-zero records in test data after tfidf

In [ ]:
test['Nzero'] = test['Feature'].apply(lambda x: np.count_nonzero(x))

In [192]:
test[test['Nzero'] == 0].groupby('Label').count()

,ID,Feature,Nzero
Label,,,
0,277,277,277
1,605,605,605
2,561,561,561
3,576,576,576
4,304,304,304


## Create a test2 after filtering all-zero records

In [ ]:
test2 = test[test['Nzero'] != 0]

In [84]:
len(test2)

1677

In [ ]:
test2.head(5)

## Download after-processing train and test dataset to local, save time for future use

## Save

In [83]:
x_train = np.array(train['Feature'].values.tolist())
y_train = train['Label']

In [87]:
x_test = np.array(test2['Feature'].values.tolist())
y_test = test2['Label']

In [88]:
np.save('x_train.npy', x_train)
np.save('y_train.npy', y_train)
np.save('x_test.npy', x_test)
np.save('y_test.npy', y_test)

## Load

In [89]:
num = 10000

In [90]:
X_train = np.load('x_train.npy')[0:num]

In [147]:
Y_train = np.load('y_train.npy')[0:num]
X_test_true = np.load('x_test.npy')
Y_test_true = np.load('y_test.npy')

In [148]:
# Y_train = pd.DataFrame(Y_train)[0]
# Y_test = pd.DataFrame(Y_test)[0]

In [149]:
Y_train

array([3, 3, 3, ..., 4, 2, 2])

## Split dataset to training and validation

In [150]:
from sklearn.model_selection import train_test_split

In [168]:
x_train, X_test, y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [169]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [170]:
y_train = pd.DataFrame(y_train)[0]
y_val = pd.DataFrame(y_val)[0]

## One-hot processing for labels

In [171]:
# one-hot，5 category
y_labels = list(y_train.value_counts().index)
#y_labels = np.unique(y_train)
le = preprocessing.LabelEncoder()
le.fit(y_labels)
num_labels = len(y_labels)
y_train = to_categorical(y_train.map(lambda x: le.transform([x])[0]), num_labels)
y_val = to_categorical(y_val.map(lambda x: le.transform([x])[0]), num_labels)

In [179]:
1677./4000

0.41925

In [184]:
print('从”Train.csv“中取出10000个data，分为Train, Val, Test三份：')
print('     Train: ', x_train.shape[0], '用于训练')
print('     Val: ', x_val.shape[0], '用于训练过程中validation')
print('     Test:', X_test.shape[0], '训练结束后用于检测')
print('\n')
print('从"Test.csv"中取出4000个data，tfidf处理后有60%data的全部feature都是0，过滤掉了这部分后剩下40%')
print('     True Test: ', X_test_true.shape[0], '训练结束后用于检测')

从”Train.csv“中取出10000个data，分为Train, Val, Test三份：
     Train:  6400 用于训练
     Val:  1600 用于训练过程中validation
     Test: 2000 训练结束后用于检测


从"Test.csv"中取出4000个data，tfidf处理后有60%data的全部feature都是0，过滤掉了这部分后剩下40%
     True Test:  1677 训练结束后用于检测


# Model Training

## Model 1: Provided by Prof Liu
* have converted to Keras platform
* not able to up and run on my local laptop by limited resouce

In [ ]:
# Previous
model = Sequential()
model.add(Embedding(x_train.shape[0], 3000, input_shape=(x_train.shape[1],)))
model.add(Convolution1D(128, 5, activation='relu'))
model.add(MaxPool1D(5, 5,padding='same'))
model.add(Convolution1D(128, 5, activation='relu'))
model.add(MaxPool1D(5, 5,padding='same'))
model.add(Convolution1D(128, 5, activation='relu'))
model.add(MaxPool1D(35, 35,padding='same'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=1000,
          epochs=10,
          validation_data=(x_val, y_val))
# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)
#l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
#l_pool1 = MaxPooling1D(5)(l_cov1)
#l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
#l_pool2 = MaxPooling1D(5)(l_cov2)
#l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
#l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
#l_flat = Flatten()(l_pool3)
#l_dense = Dense(128, activation='relu')(l_flat)
#preds = Dense(len(macronum), activation='softmax')(l_dense)

## Model 2: MLP, Multilayer Perceptron
* reference: https://zhuanlan.zhihu.com/p/29201491
* a simple NN model
* the only one that can be run on my laptop
* have best result as train_acc: 0.9798, val_acc: 0.8138, test_acc: 0.8095

In [186]:
# MLP
model = Sequential()
model.add(Dense(1024, input_shape=(x_train.shape[1],), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=500,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 6400 samples, validate on 1600 samples
Epoch 1/10
6400/6400 [==============================] - 17s 3ms/step - loss: 1.1760 - acc: 0.5619 - val_loss: 0.7521 - val_acc: 0.7763
Epoch 2/10
6400/6400 [==============================] - 12s 2ms/step - loss: 0.4052 - acc: 0.8887 - val_loss: 0.6043 - val_acc: 0.8231
Epoch 3/10
6400/6400 [==============================] - 12s 2ms/step - loss: 0.1903 - acc: 0.9408 - val_loss: 0.6813 - val_acc: 0.8181
Epoch 4/10
6400/6400 [==============================] - 12s 2ms/step - loss: 0.1333 - acc: 0.9589 - val_loss: 0.7499 - val_acc: 0.8125
Epoch 5/10
6400/6400 [==============================] - 12s 2ms/step - loss: 0.1047 - acc: 0.9688 - val_loss: 0.7577 - val_acc: 0.8188
Epoch 6/10
6400/6400 [==============================] - 12s 2ms/step - loss: 0.0861 - acc: 0.9730 - val_loss: 0.7879 - val_acc: 0.8156
Epoch 7/10
6400/6400 [==============================] - 13s 2ms/step - loss: 0.0826 - acc: 0.9761 - val_loss: 0.8265 - val_acc: 0.8181
Epoch 8

## Test result
* one test dataset is from train.csv
* one test is from test.csv
* two results have large difference

In [162]:
from sklearn.metrics import accuracy_score

In [163]:
accuracy_score(model.predict_classes(X_test), Y_test)

0.8095

In [164]:
accuracy_score(model.predict_classes(X_test_true), Y_test_true)

0.24090638044126417

## Model 3: CNN model
* not able to run on local machine....

In [ ]:
MAX_SEQUENCE_LENGTH = 19607

In [ ]:
# CNN
model = Sequential()
model.add(Embedding(x_train.shape[0], 3000, input_shape=(x_train.shape[1],)))
model.add(Convolution1D(256, 3, padding='same'))
model.add(MaxPool1D(3,3,padding='same'))
model.add(Convolution1D(128, 3, padding='same'))
model.add(MaxPool1D(3,3,padding='same'))
model.add(Convolution1D(64, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(BatchNormalization()) # (批)规范化层
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=32,
          epochs=15,
          validation_data=(x_test, y_test))

## Model 4: LSTM model
* not able to run on local machine....

In [ ]:
# LSTM
model = Sequential()
model.add(Embedding(x_train.shape[0], 300, input_length=19607))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.1))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=1000,
          epochs=10,
          validation_data=(x_test, y_test))